# Linear Classification

Reviewing some linear classification concepts. Here, I will use linear classification on the CIFAR-10 images. 

### Predicting classes

Our linear classifier looks as follows:

$$f(x_i, Wb) = Wx_i + b$$

where $x_i$ are our inputs (images) and W the weights and b the biases.


### Note on matrix shape

Note that the CIFAR-10 has 10 classes. Therefore, for a given image, our function should give the probabilities that it belongs to each of these categories. 

Note that these images will be flatten. 

In [15]:
## Get One batch of CFAR 10 data set

def load_cfar10_batch(cifar10_dataset_folder_path, batch_id):
    """
    Load a batch of the dataset
    """
    with open(cifar10_dataset_folder_path + '/data_batch_' + str(batch_id), mode='rb') as file:
        batch = pickle.load(file, encoding='latin1')

    features = batch['data'].reshape((len(batch['data']), 3, 32, 32)).transpose(0, 2, 3, 1)
    labels = batch['labels']

    return features, labels

In [16]:
data = load_cfar10_batch('../Udacity/deep_learning/projects/image-classification/cifar-10-batches-py',
                 1)

In [20]:
features = data[0]
labels = data[1]